
# **ATENÇÃO**
A lista deverá ser entregue tanto no formato .pdf e em .py ou .ipynb


# **Rotação de uma Imagem**
Uma opção seria utilizar a função de rotação do OpenCV. A ideia do exercício é não utilizar esta função, mas sim implementá-la manualmente (para compreender o que está ocorrendo nos bastidores). Para realizar este exercício, você deve preencher as lacunas da função escrita no código em anexo. Esta função tem como entrada:

.uma imagem (img),

.o ângulo de rotação (angulo),

.o centro de rotação (centro),

e devolve a imagem rotacionada.

In [3]:
import cv2 as cv
import sys
import numpy as np

In [5]:
def my_rotation(img, angulo, centro):
    # nessa primeira parte, vamos definir a transformação que leva a posicao dos pixels da imagem original
    # para a posicao dos pixels do imagem rotacionada.

    # a primeira matriz de translação muda a origem das coordenadas do canto da imagem para o centro da imagem
    matriz_translacao = np.array([[1, 0, -centro[0]], [0, 1, -centro[1]], [0, 0, 1]])
    # a matriz de rotacao aplica a rotacao em torno da origem
    matriz_rotacao = np.array([[np.cos(np.radians(angulo)), -np.sin(np.radians(angulo)), 0], [np.sin(np.radians(angulo)), np.cos(np.radians(angulo)), 0], [0, 0, 1]])
    # a composicao coloca todas as matrizes em uma só: aplica a translacao (muda a origem), rotaciona, volta para a origem anterior
    matriz_composicao = np.dot(np.dot(matriz_translacao, matriz_rotacao), np.linalg.inv(matriz_translacao))

    # criar imagem rotacionada em preto, com mesmas dimensões da original
    height, width = img.shape[:2]
    rotated_image = np.zeros((height, width, 3), np.uint8)

    # o próximo passo é percorrer cada pixel da nova imagem e verificar qual é o pixel correspondente na imagem original
    m_comp_inv = np.linalg.inv(matriz_composicao)
    for linr in range(height):
        for colr in range(width):
            pos_rot = np.array([linr, colr, 1])
            pos_orig = np.dot(m_comp_inv, pos_rot)
            lin = round(pos_orig[0]); col = round(pos_orig[1])
            if (lin >=0 and lin < height) and (col >= 0 and col < width):
                # opa, é um pixel pertencente à imagem original...
                rotated_image[linr, colr] = img[lin, col]

    return rotated_image

In [6]:
img = cv.imread('Palazzo_Farnese_Fassade.jpg')
height, width = img.shape[:2]
centro = (height/2, width/2)
angulo = np.pi/2
my_rotated_image =  my_rotation(img, angulo, centro)

In [4]:
cv.imshow('Display window', my_rotated_image)
cv.waitKey(0); cv.destroyAllWindows()

: 

Você pode comparar seu resultado com o função do Open CV.

In [7]:
# Obter o centro da imagem
height, width = img.shape[:2]
center = (width/2, height/2)

# Definir a matriz de rotação
M = cv.getRotationMatrix2D(center, 30, 1)

# Aplicar a rotação na imagem
rotated_img = cv.warpAffine(img, M, (width, height))

cv.imshow('Display window', rotated_img)
cv.waitKey(0); cv.destroyAllWindows()

: 

# **Transformação Projetiva de uma Imagem**
Escreva uma função que tenha uma como entrada:

. uma imagem (img),

. uma matriz de uma transformação projetiva (T),

e devolva a imagem transformada.

A ideia é semelhante ao exercício 1, mas ao invés da rotação, temos uma transformação projetiva (homografia).

Teste a sua função, faça a transformação projetiva com apenas um ponto de fuga no eixo $x$ (digamos, o ponto $(2000,0)$ em coordenadas $x,y$), da imagem do Palazzo Farnese Fassade (em anexo).

Você pode usar outra imagem de sua preferência, caso queira. Não se esqueça de trabalhar com coordenadas homogênas.

In [7]:
def my_transform(img, T):
# Escreva seu código aqui
# criar imagem tranformada, com mesmas dimensões da original
    height, width = img.shape[:2]
    transformed_img = np.zeros((height,width,3), np.uint8)

    # calcula a matriz inversa de T
    T_inv = np.linalg.inv(T)

    # itera sobre cada pixel na imagem transformada
    for lint in range(height):
        for colt in range(width):
            # obtém as coordenadas originais, multiplicando pela inversa
            coords = np.dot(T_inv, [colt, lint, 1])
            if coords[2] != 0: # verificando se o ponto é impróprio
                x = int(coords[0] / coords[2])
                y = int(coords[1] / coords[2])

                # verifica se as coordenadas originais estão dentro dos limites da imagem
                if 0 <= x < width and 0 <= y < height:
                    # copia o pixel da imagem original para a imagem transformada
                    transformed_img[lint, colt] = img[y, x]

    return transformed_img

In [8]:
img = cv.imread('Palazzo_Farnese_Fassade.jpg')
T = np.array([[1,0,0],
              [0,1,0],
              [0.0005,0,1]])
my_tranform_img =  my_transform(img, T)

In [9]:
cv.imshow('Display window', my_tranform_img)
cv.waitKey(0); cv.destroyAllWindows()

# **Estimação da Transformação Projetiva**
**Crie uma função**

Escreva uma função que tenha uma como entrada:

. Quatro pontos “fonte” no plano projetivo, possivelmente impróprios,

. Quatro pontos “destino” no plano projetivo, possivelmente impróprios.

E devolva a transformação projetiva $T: \mathbb{R}\mathbb{P}^2 \rightarrow \mathbb{R}\mathbb{P}^2$ que leva os pontos “fonte” nos pontos
“destino”.

In [10]:
def my_estimation(lst1, lst2):
    # inicializando uma matriz de zeros com 8 linhas e 9 colunas
    A = np.zeros((8, 9))
    
    # iterando sobre as listas de pontos
    for i in range(4):

        x, y = lst1[i] # ponto da lista 1

        x2, y2 = lst2[i] # ponto correspondente da lista 2
        
        # preenchendo a matriz A
        A[2 * i] = [-x, -y, -1, 0, 0, 0, x2 * x, x2 * y, x2]
        A[2 * i + 1] = [0, 0, 0, -x, -y, -1, y2 * x, y2 * y, y2]
    

    _, _, V = np.linalg.svd(A)
    
    # obtendo t apartir da ultima linha de V
    t = V[-1, :] / V[-1, -1]
    
    # obtendo a matriz desejada redimensionando t
    estimation_T = t.reshape(3, 3)

    return estimation_T

Encontre a transformação projetiva que leva o campo de futebol da imagem
do gol no jogo Vasco contra Flamengo (em anexo) em um retângulo correspon-
dente ao campo de futebol visto de cima em projeção ortogonal (em anexo, mas
fique a vontade para usar outra). Para isso, utilize a sua função my_estimation.

Use esta projeção para desenhar o campo
de futebol visto de cima junto com a posição do Leo Pelé, do goleiro e de um
jogador do Flamengo. Para isso, utilize a sua função my_transform.

In [11]:
# Escreva seu código aqui
vasco_flamengo_img = cv.imread('Vasco_Flamengo.jpg')
campo_fut_img = cv.imread("Campo_Futebol.webp")

# escolhendo par de pontos específicos das imagens(pontos da pequena área e um da grande área)
projetiva_pontos = [(2,339), (90,207), (204,339), (614,145)]
campo_pontos = [(42,258), (41,155), (72,259), (134,94)]

# obtendo a transformação a partir dos pontos
matriz_tranformacao = my_estimation(projetiva_pontos,campo_pontos)

goleiro = (95,250,1) # ponto do goleiro
jogador = (498,293,1) # ponto de um jogador do flamengo

# Aplicando a transformação nos pontos para levar a posição deles na visão de cima
goleiro_transform = matriz_tranformacao @ goleiro
jogador_transform = matriz_tranformacao @ jogador

# como os pontos estão em coordenadas homogêneas temos que mudar para coordenadas cartesianas
goleiro_transform = (int(goleiro_transform[0]/goleiro_transform[2]), int(goleiro_transform[1]/goleiro_transform[2]))
jogador_transform = (int(jogador_transform[0]/jogador_transform[2]), int(jogador_transform[1]/jogador_transform[2]))

# desenhando os pontos na imagem
cv.circle(campo_fut_img, goleiro_transform, 5, (0, 0, 255), -1)
cv.circle(campo_fut_img, jogador_transform, 5, (0, 0, 255), -1)

cv.imshow('Display window', campo_fut_img)
cv.waitKey(0); cv.destroyAllWindows()


Compare seu resultado com o implementado no OpenCV (cv.warpAffine)

# **EXTRA**
# You are Fake News ... or maybe not.
Muitas vezes, os torcedores apaixonados pelo time desconfiam do resultado do VAR e das decisões de impedimento, como foi o caso do vídeo. Teste sua função na imagem da partida entre Union e Boca Juniors (em anexo) e verifique se o método mostrado no vídeo (https://www.instagram.com/papodeboleiros/reel/CvAhQazvuhu/) produz o mesmo resultado de impedimento que a função implementada (basta fazer a transformação na imagem com as "retas paralelas" do vídeo e verificar se de fato são paralelas). Comente sobre o resultado da comparação. Você acredita que este método seja sempre correto? Caso contrário, forneça um contraexemplo onde ele falhe (se possível, com uma imagem).


In [12]:
# Escreva seu código aqui

# My VAR-lidation
Utilizando suas funções my\_estimation e my\_transform, faça uma função que tem como entrada:

. Duas imagens, uma do jogo e uma do campo (img1,img2)

. Uma lista de pontos "fonte" e "destino"(lst1,lst2)

. Os pixels  relativos aos dois jogadores que estão sendo analisados (p1,p2)

E devolve a imagem do campo com uma linha marcada de ambos os jogadores, semelhante com as análises do VAR.

Perceba que basta desenhar uma linha (pintando todos os pixels com a mesma coordenada em x da mesma cor) quando a imagem do jogo estiver em projeção ortogonal e depois usar a inversa da matriz de projeção para voltar ao formato original.

Observação: a imagem provavelmente sofrerá perda de qualidade neste processo. Se desejar manter a qualidade, você poderá realizar esse processo apenas em uma imagem com a linha inserida e adicioná-la depois de transformada na imagem original.

In [13]:
# Escreva seu código aqui